## Improve Efficiency with Machine Learning and Caesar

In [ ]:
from panoptes_client import Panoptes, Project, Subject, SubjectSet
import pandas as pd
import os
import getpass
import glob

Trained machine learning models can take up a lot of disk space and can be a computationally intensive process. Using machine learning to predict classifications for specific subjects not the sort of analysis that _Caesar_ can run by itself (even if you did add the code to the `aggregation_for_caesar` repository. 

However, that doesn't mean that we can't use machine learning models and Caesar _together_ to make our classification workflow more efficient. For example you might want to use a machine learning model to predict the class of every subject in a subject set and then use less stringent retirement criteria if the machnie and human classifiers agree.

As a concrete example, we might say that if the first three volunteers all agree with the machine prediction, then we can retire the suject but we would demand that seven volunteers view and classify the subject otherwise.

There are lots a couple of easy ways to couple a machine learning model with your Zooniverse project and we'll quickly demo them here.

1. The first option we have is to preclassify all of our subjects and add a **hidden subject metadatum** with the mahine learning score that Caesar can recongise and use to execute specific retirement rules.
2. The second option is to make machine learning predictions for subjects **after** they have been classified and make an "on the fly" retirement decision. The second approach is much more complicated, but it does mean that you can benefit from the latest version of your machine learning model and you can compute very complicated retirement criteria that are difficult or impossible to express as Caesar rules. 

### Option 1: Make machine learning prediction for all subjects "up front"
If you have a trained machine learning model and a pool of data, then it's straightforward to predict a machine score for all of your subjects and add a new hidden metadatum to each one.

We've already trained a simple CNN to predict the class of a variable star lightcurve by looking at the same images the volunteers see. We can load a table of its predictions for our subjects.

In [ ]:
dataDirectory = ??
predictions = pd.read_csv(os.path.join(dataDirectory, "predictions.csv")).set_index("image")
predictions

Now let's create a new subject set (as we've done many times now) and add the machine learning prediction as a hidden metadatum. We can call that metadatum whatever we like, so how about `#ml_prediction`? We'll simplify things by defining the machine prediction for a subject to be the category with highest score, so for the zeroth row, our machine prediction is "Pulsator". Rather than storing the prediction as a word though, we'll map it to its corresponding position in the list of options for task 1 of our project. Recall that that order is:
```python
["Pulsator", "EA/EB type", "EW type", "Rotator", "Unkown", "Junk"]
```

In [ ]:
questionOrder = ["Pulsator", "EA/EB type", "EW type", "Rotator", "Unknown", "Junk"]

Let's start with all the parts we've seen before.

In [ ]:
username = input("Enter Panoptes Username:")
password = getpass.getpass("Enter Panoptes Password:")

In [ ]:
panoptesClient = Panoptes.connect(username=username, password=password)

In [ ]:
projectId = ??
project = Project.find(projectId)

Here's where we create our new subject set and give it a name and link it to our project.

In [ ]:
subjectSet = SubjectSet()
subjectSet.display_name = "Machine Learning Demo Set"
subjectSet.links.project = project
response = subjectSet.save()

In [ ]:
imageFiles = glob.glob(os.path.join(dataDirectory, "demoset_smallSample_1/*jpg"))
imageFiles

We'll use those files to create a list of new `Subjects` and add some metadata. First let's process the machine predictions like we discussed above.

In [ ]:
imagePredictions = []
for imageFile in imageFiles:
    imageRow = int(os.path.basename(imageFile)[:-4])
    imagePredictions.append(
        (
            imageRow,
            predictions.iloc[imageRow].idxmax(),
            questionOrder.index(predictions.iloc[imageRow].idxmax()),
        )
    )
imagePredictions

In [ ]:
newSubjects = []

for imageFile, imagePrediction in zip(imageFiles, imagePredictions):

    newSubject = Subject()
    newSubject.links.project = project
    newSubject.add_location(imageFile)
    newSubject.metadata = {
        "Origin": "Python ML demo",
        "image": os.path.basename(imageFile),
        "#ml_prediction": imagePrediction[2],
    }
    newSubject.save()

    newSubjects.append(newSubject)

And finally, we assign our newly uploaded `Subject`s to the `SubjectSet` we already created.

In [ ]:
subjectSet.add(newSubjects)

Now we just have to **set up appropriate retirement rules in the Caesar UI**.

### Option 2: Make machine learning prediction for each subject "on the fly"

#### External reducers
Instead Caesar defines the concept of an _External_ reducer. When you set up an external reducer in the Caesar UI, the system will send the results from any associated extractors to an HTTP endpoint that you specify. You can then receive those extract data and process them appropriately. 

#### Placeholder reducers
Once processing is complete, you need a way to send the results back to Caesar. The system defines the concept of a _Placeholder_ reducer for this purpose. Your external reducer should send its answer back to the placeholder reducer, allowing Caesar to process any associated rules or effects.

#### SQS Reducers
The one downside of a basic external reducer is that you need to run some sort of webserver that can listen out for messages from Caesar and process them when they arrive. This can be a barrier to entry for some research teams, so Caesar defines one more reducer category, the _SQS Reducer_. 

_SQS_ stands for "Simple Queue Service". It's a facility provided by Amazon Web Services that maintains a queue of messages in the cloud. Caesar sends messages to this queue and your code can consume them. All the web hosting is handled by Amazon, so you don't have to worry.

We'll set up an SQS reducer in the Caesar UI (see the recording that goes with this notebook) to send message extracts to us. Then we can use a specially written client to grab those messages from the queue and process them.

In [ ]:
from ML_Demo import SQSClient

**IMPORTANT**: Note that for you to use the SQS reducer facility you will need to set up an Amazon AWS account, set up a queue and get a set of access credentials. This process is free and reasonably straightforward, but beyond the scope of this tutorial. Once you've done that you can export your credentials to your computer's environment like this.

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = ??
os.environ["AWS_SECRET_ACCESS_KEY"] = ??

Every SQS queue has a unique URL that you can use to connect to it and retrieve messages. You must pass this URL to the `SQSClient` constructor when you instantiate the SQS client.

In [ ]:
queueUrl = "https://sqs.us-east-1.amazonaws.com/927935712646/CaesarEuclid"
sqs = SQSClient(queueUrl)

The client provides a `getMessages` method that will retrieve a batch of messages from the queue and (optionally) delete them from the queue. The method returns three objects. The first is the set of messages that were received, with any duplicates removed, the second is the raw set of messages before deduplication and the final object is a list of unique message IDs for the messages that were received.

In [ ]:
messages, receivedMessages, receivedMessageIds = sqs.getMessages(delete=True)

We can examine our messages more easily if we convert to a `pandas.DataFrame`.

In [ ]:
messagesFrame = pd.DataFrame(messages)
messagesFrame

For every classification we get a row of data corresponding. The information about the volunteer's answer is in the `data` column. Since we set Caesar to use a question extractor the format of each entry can be interpreted as:
```python
{ '<answer>' : <number_of_matching_responses> } 
```
So fo example in row 0 of the table we can see that for subject 52665118, the classification contained one response for the answer `'0'`, which corresponds to a Pulsator.

If we have a (partially) trained machine learning model, we could ask it to make its own predictions for each of the subjects we just got classifications for, but for now, let's just look up the classification (if it exists) in our table of predictions. We can use the `subject_id` field in our message table to get the metadata for each subject and then try to retrieve its machnie learning prediction.

In [ ]:
machineMatchesHuman = []
for _, (subjectId, data) in messagesFrame[["subject_id", "data"]].iterrows():
    answer = next(iter(data.keys()))
    if len(answer) == 0:
        machineMatchesHuman.append(False)
        continue
    answer = int(answer)
    try:
        subject = Subject.find(subjectId)
    except:
        machineMatchesHuman.append(False)
        continue
    if "image" in subject.metadata:
        lookup = int(subject.metadata["image"][:-4])

    if lookup in predictions.index:
        print(
            subjectId,
            subject.metadata["image"][:-4],
            predictions.loc[lookup].idxmax(),
            questionOrder[answer],
        )
        machineMatchesHuman.append(
            answer == questionOrder.index(predictions.loc[lookup].idxmax())
        )
        continue
    machineMatchesHuman.append(False)

machineMatchesHuman

We decide to retire all subjects for which the machine and human predictions match. Now we need to send a message back to Caesar with our decision for the subjects we want to retire. If we don't want to do anything we don't have to send a message. We can use the Panoptes Python API send a message back to the placeholder reducer we set up in the Caesar UI. Here's how we do it.

In [ ]:
pan = Panoptes(login="interactive", redirect_url='https://caesar.zooniverse.org/auth/zooniverse/callback')

In [ ]:
response = pan.put(endpoint="https://caesar.zooniverse.org", 
                   path=f"workflows/{??}/subject_reductions/test_placeholder", 
                   json={
                       'reduction': {
                           'subject_id': int(messagesFrame.subject_id.iloc[-1]),
                           'data': {"retire": True}
                       }
                   })

If the reduction was successfully submitted, Caesar will send a `dict` in response.

In [ ]:
response

Success!

#### Aside: Avoid entering credentials
To avoid having to enter credentials to send reductions back to Caesar, you can register with the Panoptes authentication system and get a _Client ID_ and a _Client Secret_. These are just special strings of unguessable characters that the Panoptes Python API can use instead of your usual credentials to authenticate. 

To get your _Client ID_ and _Client Secret_  visit [https://panoptes.zooniverse.org/oauth/applications](https://panoptes.zooniverse.org/oauth/applications) and click on _New Application_. Once you have those details you can export them to your computer's environment just like you did for the Amazon credentials, but with different names.

In [ ]:
os.environ["PANOPTES_CLIENT_ID"] = ??
os.environ["PANOPTES_CLIENT_SECRET"] = ??

### Filtering Subjects Using Machine Learning
Although machine learning algorithms are very good at performing **specific tasks**, there are some things that human beings still tend to do better at. For example, human beings are much more likely to spot unusual or unexpected features in images or other types of subjects.

However, there are many datasets (with more arriving every day) was that are simply too large to be processed by human beings, even using a citizen science approach.

Machine learning can help here by filtering out subjects that are "not interesting". Such subjects are typically very common in the datasets that were used to train the machnine learning models and are therefore very easily and confidently classified using those models.

Commonly cited examples of how machine learning has been used to filter subjects that are shown to volunteers on Zooniverse are ecolology-focussed "camera trap" projects. Volunteers are asked to identify any animals they see in the images they are shown. Machine learning models detect "empty" images very accurately and it is not useful for volunteers to classifiy images with no animals in them. Machine learning can be used very effectively to remove empty images from Zooniverse subject sets to let volunteers focus on classifying animals.

We'll use an example from our SuperWASP CNN. We'll select only those lightcurves for which our model is "confused". We'll define "confusing" images as those for which the machine learning algorithm outputs a response greater than 0.4 for more than one category.

For this demonstration, we've preselected those subjects and you can find the images in the `data/demoset_confused` folder. 

In [ ]:
confusedPredictions = pd.read_csv(os.path.join(dataDirectory, "confused_predictions.csv")).set_index("image")

In [ ]:
confusedPredictions

Let's make a quick plot of some of these "confusing" images.

In [ ]:
from ML_Demo import plotConfusedBatch

In [ ]:
plotConfusedBatch(confusedPredictions.iloc[:, :6], os.path.join(dataDirectory, "demoset_confused"))

I think we can agree that the answers certainly aren't obvious, but as humans (maybe experts) we can probably get all of them right. Let's use our normal techniques to make a new "confused" subject set and add our confused subjects to it.

In [ ]:
subjectSet = SubjectSet()
subjectSet.display_name = "Confused Demo Set"
subjectSet.links.project = project
response = subjectSet.save()

In [ ]:
newSubjects = []

for image, imagePrediction in confusedPredictions.iloc[:, :6].iterrows():

    newSubject = Subject()
    newSubject.links.project = project
    newSubject.add_location(os.path.join(os.path.join(dataDirectory, "demoset_confused"), f"{image}.jpg"))
    newSubject.metadata = {
        "Origin": "Python ML demo",
        "image": os.path.basename(imageFile),
        "#ml_prediction": ", ".join(imagePrediction),
    }
    newSubject.save()

    newSubjects.append(newSubject)

### Active Learning

We're now in a position to implement a toy demonstration of a technique called "active learning". In active learning, model predictions are used to select subjects that are likely to provide the most useful information to improve the model's performance if they were labelled. Our "confusing subjects" selection probably isn't exactly the right approach, but it's a reasonable attempt.

Now that we've created a new subject set with confusing images, let's create a special "Advanced" workflow to process them. We can use Caesar to send classifications from that workflow to our SQS queue once they're classified and we can use those classifications to retrain our model.

In [ ]:
messages, receivedMessages, receivedMessageIds = sqs.getMessages(delete=True)
messagesFrame = pd.DataFrame(messages)
messagesFrame

Once we collect enough classifications from our "Advanced" workflow we can retrain our model, select a new set of confusing subjects, and use them to create another subject set for the "Advanced" workflow. Repeating this cycle over and over again is the basis of active learning with Zooniverse.